In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn.functional import relu
from torch.utils.data import DataLoader
from torchvision import datasets

Change the path below to where your "fer2013" folder is
Example: If fer2013 is located in the Downloads folder, the path looks something like below

In [18]:
dataset_root_folder = r'C:\Users\nguye\Downloads'

In [19]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.FER2013(root=dataset_root_folder)